In [177]:
import numpy as np
import pandas as pd
import os
import rqdatac as rq

In [17]:
import missingno as msno

In [178]:
rq.init('license', 'QhhTnnWjN5eWgeBzXChE-d4iqymtu4-6XIBIjcO4Oh339_i0W1LifdY980vuXMEq-Zxz03HTXeItx7cK-704N0Go8i7DERQjkXQIirqt6l4ZTVJIcJbC1Vh62p5npTupQN0F9SyXTIoE1UQPacRqhHUSWHQm4yNFQW2nHffVVlY=Th4zB2-UYw5Q76xBPpPrLCKIjzpVid9Wogpi8QP9Ju8M7ZRUXzA9V6TmKCWOlCrTPBWnA8HAjEL-3hM55cAac-hzVADGoxpfV4tHA79mMsjOCREOTXzQH6rqcqUuEDuzShSWFB069fRSetK3yzgeeRdqocHmJ4ATA3a3pCcAVpA=')

C:\Users\jiyic\anaconda3\lib\site-packages\rqdatac\client.py:197: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)
C:\Users\jiyic\anaconda3\lib\site-packages\rqdatac\client.py:236: UserWarning: Your account will be expired after  209 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


In [179]:
rq.user.get_quota()

{'bytes_used': 225476352,
 'bytes_limit': 209715200.0,
 'remaining_days': 209,
 'license_type': 'TRIAL'}

In [11]:
import re 

stock_path = './data/stock_data/'
stock_names = os.listdir(path=stock_path)

format_stock_names = []
for name in stock_names:
    format_stock_names.append(name[:-4].strip().upper())

In [12]:
def normalize_code(symbol, pre_close=None):
    """
    归一化证券代码

    :param code 如000001
    :return 证券代码的全称 如000001.XSHE
    """
    XSHG = 'XSHG'
    SSE = 'XSHG'
    SH = 'XSHG'
    XSHE = 'XSHE'
    SZ = 'XSHE'
    SZE = 'XSHE'
    if (not isinstance(symbol, str)):
        return symbol

    if (symbol.startswith('SZ') and (len(symbol) == 8)):
        ret_normalize_code = '{}.{}'.format(symbol[2:8],SZ)
    elif (symbol.startswith('SH') and (len(symbol) == 8)):
        ret_normalize_code = '{}.{}'.format(symbol[2:8], SH)
    elif (symbol.startswith('00') and (len(symbol) == 6)):
        if ((pre_close is not None) and (pre_close > 2000)):
            # 推断是上证指数
            ret_normalize_code = '{}.{}'.format(symbol, SH)
        else:
            ret_normalize_code = '{}.{}'.format(symbol, SZ)
    elif ((symbol.startswith('399') or symbol.startswith('159') or \
        symbol.startswith('150')) and (len(symbol) == 6)):
        ret_normalize_code = '{}.{}'.format(symbol, SH)
    elif ((len(symbol) == 6) and (symbol.startswith('399') or \
        symbol.startswith('159') or symbol.startswith('150') or \
        symbol.startswith('16') or symbol.startswith('184801') or \
        symbol.startswith('201872'))):
        ret_normalize_code = '{}.{}'.format(symbol, SZ)
    elif ((len(symbol) == 6) and (symbol.startswith('50') or \
        symbol.startswith('51') or symbol.startswith('60') or \
        symbol.startswith('688') or symbol.startswith('900') or \
        (symbol == '751038'))):
        ret_normalize_code = '{}.{}'.format(symbol, SH)
    elif ((len(symbol) == 6) and (symbol[:3] in ['000', '001', '002', 
                                                 '200', '300'])):
        ret_normalize_code = '{}.{}'.format(symbol, SZ)
    else:
        print(symbol)
        ret_normalize_code = symbol

    return ret_normalize_code

In [14]:
normalized_code = [normalize_code(name) for name in format_stock_names] # length = 4560
normalized_code[0]

'600000.XSHG'

Set time period for factor data = [20101231, 20201231] 

Non-trading days are not counted in the dataframe.

保证非交易日没有纳入因子数据后，我们要在每个选股日剔除当天的ST股、停牌股（所有还是按比例？）、已退市和上市不满3(?)年的股票。

交易日的选择未定（每月月底？），如果按十年内ST/停牌出现的比例剔除ST、停牌、退市股或许存在前视偏差(look-ahead bias)

这里先剔除所有ST股

In [16]:
# 检测回测时间内是否有ST退市股票
non_st_stock = rq.is_st_stock(normalized_code, '20100101','20201231')
# get column values i.e. stock code sorted
non_st_stock = non_st_stock.reindex(sorted(non_st_stock.columns), axis=1)

C:\Users\jiyic\anaconda3\lib\site-packages\rqdatac\validators.py:224: UserWarning: invalid order_book_id: 600849.XSHG
  warnings.warn("invalid order_book_id: {}".format(ob))


In [107]:
st_count = [non_st_stock[code].value_counts() for code in non_st_stock.columns]
# stocks used to be ST during our backtesting period
st_df = pd.DataFrame(st_count)[~st_df[True].isnull()].sort_values(by=True,ascending=False)

In [112]:
st_df

,False,True
600817.XSHG,NaN,2674.0
600091.XSHG,79.0,2595.0
600301.XSHG,234.0,2440.0
600145.XSHG,480.0,2194.0
600234.XSHG,497.0,2177.0
...,...,...
300104.XSHE,2643.0,31.0
300431.XSHE,2643.0,31.0
300216.XSHE,2643.0,31.0
600185.XSHG,2645.0,29.0


545 out of 4560 stocks were once warned ST, so we consider kick them out from our price data.

600849.XSHG changed its ticker code, so it is automatically kicked out of the pool.

In [119]:
# check the number of suspended days for each stock
# 每只股票停牌天数
check_suspend= rq.is_suspended(normalized_code, '20100101','20201231', market='cn')
check_suspend = check_suspend.reindex(sorted(check_suspend.columns), axis=1)
suspended_count = [check_suspend[code].value_counts() for code in check_suspend.columns]
# stocks used to have suspended days during our backtesting period
suspend_df = pd.DataFrame(suspended_count)

In [132]:
suspend_df = suspend_df[~suspend_df[True].isnull()].sort_values(by=True,ascending=False)
suspend_df

,False,True
000693.XSHE,859.0,1815.0
600145.XSHG,1343.0,1331.0
000670.XSHE,1417.0,1257.0
000038.XSHE,1447.0,1227.0
600817.XSHG,1484.0,1190.0
...,...,...
002903.XSHE,2673.0,1.0
601611.XSHG,2673.0,1.0
603798.XSHG,2673.0,1.0
603839.XSHG,2673.0,1.0


3065 out of 4560 stocks were suspended at least for one day in the period.

### Unify stock codes that are used in Alphalens

In [166]:
st_stocklist = st_df.index.to_list()
# subset the stocks with more than 365 days being suspended
suspend_stocklist = suspend_df[suspend_df[True]>365].index.to_list()

In [167]:
no_st_code = list(set(normalized_code) - set(st_stocklist))
less_1k_suspend_code = list(set(normalized_code) - set(suspend_stocklist))
codes = list(set(no_st_code) & set(less_1k_suspend_code))

In [168]:
len(codes)

3957

In [173]:
backtest_price_data = pd.read_hdf('10-year non-ST price data.h5')
backtest_price_data = backtest_price_data.fillna(method='ffill')
backtest_price_data.columns = [normalize_code(name) for name in backtest_price_data.columns]

# get the intersection of codes between factor data and price data
codes = list(set(codes) & set(backtest_price_data.columns))

In [172]:
len(stock_codes)

2183

### Store factor data in h5 files locally

In [180]:
# all in ttm, not lyr 全部12个月滚动指标

#市净率
pb = rq.get_factor(codes,'pb_ratio_ttm','20101221','20201231') 
#净资产收益率
roe = rq.get_factor(codes,'return_on_equity_ttm','20101221','20201231') 
#总市值
mkt_cap = rq.get_factor(codes,'market_cap_3','20101221','20201231')
#股息率
dividend_yield = rq.get_factor(codes,'dividend_yield_ttm','20101221','20201231')
#账面市值比
bm = rq.get_factor(codes,'book_to_market_ratio_ttm','20101221','20201231') 
#三日平均换手率
vol3 = rq.get_factor(codes,'VOL3','20101221','20201231') 

QuotaExceeded: Quota exceeded

In [ ]:
pb.to_hdf('pb.h5', key='factor')
roe.to_hdf('roe.h5',key='factor')
mkt_cap.to_hdf('mkt_cap.h5',key='factor')
dividend_yield.to_hdf('dividend yield.h5', key='factor')
bm.to_hdf('bm.h5',key='factor')
vol3.to_hdf('vol3.h5',key='factor')